In [18]:
# Using the bound in lemma 4.1 in the paper
# NOT constraining g(x, y) = (1 + h(x) - h(y))/2

prob = LpProblem("problem4", LpMaximize)

t = LpVariable("t", lowBound=0, upBound=1)
prob += t

n = 100
a = [k/n for k in range(0, n+1)]

ijlist = [[0, 0], [n, n], [0, n], [n, 0], [int(23*n/40), int(27*n/40)], [int(20*n/40), int(30*n/40)], [int(26*n/60), int(46*n/60)]]
#ijlist = [[int((n/10)*i), int(n/10)*j] for i in range(11) for j in range(11)]

g_vars = {(i,j): LpVariable(cat=LpContinuous, 
               lowBound=0, upBound=1, 
               name="g_{0}_{1}".format(i, j)) 
          for i in range(0, n+1) for j in range(0, n+1)}

# g(x, y) increasing in x
for j in range(0, n+1):
    for i in range(1, n+1):
        prob += g_vars[i, j] - g_vars[i-1, j] >= 0, "inceasing_constraint_{0}_{1}".format(i, j)
        
# g(x, y) decreasing in y
for i in range(0, n+1):
    for j in range(1, n+1):
        prob += g_vars[i, j] - g_vars[i, j-1] <= 0, "decreasing_constraint_{0}_{1}".format(i,j)

# dg(x, y)/dy >= g(x,y) - 1
for i in range(0, n+1):
    for j in range(0, n):
        prob += n*(g_vars[i, j+1] - g_vars[i, j]) -g_vars[i, j+1] >= -1, "derivative_constraint_{0}_{1}".format(i, j)

# dg(x, y)/dx <= g(x,y) 
for j in range(0, n+1):
    for i in range(0, n):
        prob += n*(g_vars[i+1, j] - g_vars[i, j]) - g_vars[i+1, j] <= 0, "derivative_constraint2_{0}_{1}".format(i,j)

        
# # dg(x, y)/dy >= g(x,y) - 1
# for i in range(n):
#     for j in range(n):
#         prob += n*(g_vars[i, j+1] - g_vars[i, j]) >= g_vars[i+1, j]-1, "derivative_constraint_{0}_{1}".format(i, j)
# # i=n
# for j in range(n):
#     prob += n*(g_vars[n, j+1] - g_vars[n, j]) >= g_vars[n, j]-1, "derivative_constraint_{0}_{1}".format(n, j)
  

# # dg(x, y)/dx <= g(x,y) 
# for j in range(n):
#     for i in range(n):
#         prob += n*(g_vars[i+1, j] - g_vars[i, j]) <= g_vars[i,j+1], "derivative_constraint2_{0}_{1}".format(i,j)
# # j = n
# for i in range(n):
#     prob += n*(g_vars[i+1, n] - g_vars[i, n]) <= g_vars[i,n], "derivative_constraint2_{0}_{1}".format(i,n)
    
#g(1, y) - g(x, y) >= g(1, y') - g(x, y') for all x, y' > y
for i in range(0,n+1):
    for j in range(0, n+1):
        for k in range(j, n+1):
            prob += g_vars[n, j] - g_vars[i, j] >= g_vars[n, k] - g_vars[i, k]
        
        
s_vars = {(i,j,k): LpVariable(cat=LpContinuous,
                             lowBound=0,
                             upBound=1,
                             name="s_{0}_{1}_{2}".format(i,j,k))
         for i,j in ijlist for k in range(0, j+1)}

# # approximate integrals by right sums so as to get a upper bound
# for i in range(0, n+1):
#     for j in range(0, n+1):
#         for k in range(0, j+1):
#             for l in [0, i]:
#                 prob += s_vars[i,j,k] <= (1 - g_vars[l, k]
#                                            + (1/n)*lpSum(g_vars[d+1,k] for d in range(0,l))
#                                            + (1/n)*lpSum(g_vars[d+1,j] for d in range(l,i)))
                
# approximate integrals by right sums so as to get a upper bound
for i,j in ijlist:
    for k in range(0, j+1):
        for l in [0, i]:
            prob += s_vars[i,j,k] <= (1 - g_vars[l, k]
                                       + (1/n)*lpSum(g_vars[d+1,k] for d in range(0,l))
                                       + (1/n)*lpSum(g_vars[d+1,j] for d in range(l,i)))
    
# t = min{(1-a)(1-b) + (1-b)int_0^a g(x, b)dx + int_0^b min_c{g(x, c) + int_0^c g(y, x)dy + int_c^a g(y, b)dy}dx}
# where 0 <= a, b <= 1, c <= a, and g(x, y) = (1+h(x)-h(y))/2
# let s(a, b, x) = min_c{g(x, c) + int_0^c g(y, x)dy + int_c^a g(y, b)dy}
# so t = min{(1-a)(1-b) + (1-b)int_0^a g(x, b)dx + int_0^b s(a, b, x) dx}
for i, j in ijlist:
    prob += t <= ((1-a[i])*(1-a[j]) 
                   + (1-a[j])*(1/n)*lpSum(g_vars[l+1, j] for l in range(0, i))
                   + (1/n)*lpSum(s_vars[i,j,k] for k in range(0, j))) # unclear whether this is monotonic in x

In [19]:
prob.solve(solver=GUROBI())

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 573114 rows, 16373 columns and 2748645 nonzeros
Model fingerprint: 0x3579c529
Coefficient statistics:
  Matrix range     [1e-03, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-02, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 121 rows and 16055 columns (presolve time = 8s) ...
Presolve removed 121 rows and 16055 columns
Presolve time: 9.69s
Presolved: 16252 rows, 573432 columns, 2757482 nonzeros

Elapsed ordering time = 5s
Elapsed ordering time = 8s
Elapsed ordering time = 10s
Ordering time: 12.03s

Barrier statistics:
 Dense cols : 21
 AA' NZ     : 2.846e+06
 Factor NZ  : 5.417e+07 (roughly 700 MBytes of memory)
 Factor Ops : 2.781e+11 (roughly 26 seconds per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl 

1

In [20]:
value(prob.objective)

0.6679717948171288

In [21]:
value(prob.objective) + 1/(4*n)

0.6704717948171287